In [76]:
import pandas as pd
from datetime import datetime

class StockTrader:
    def __init__(self, symbol, initial_balance=100000):
        self.symbol = symbol
        self.balance = initial_balance
        self.stock_quantity = 0
        self.stock_price = 0
        self.last_generated_id = int(time.time())
        self.logs = pd.DataFrame(columns=['Symbol', 'BuyID', 'Action', 'Quantity', 'BuyPrice', 'BuyDatetime', 'SellTransactions', 'PendingQuantity', 'AverageSellPrice', 'PnL', 'Status'])

    def generate_id(self):
        unique_id = self.last_generated_id
        self.last_generated_id += 1
        return unique_id

    def buy_stock(self, quantity, price, date):
        buy_id = self.generate_id()
        buy_datetime = datetime.strptime(date, '%Y-%m-%d')
        self.balance -= quantity * price
        self.stock_quantity += quantity
        self.stock_price = price
        new_log = pd.DataFrame({'Symbol': [self.symbol], 'BuyID': [buy_id], 'Action': ['Buy'], 'Quantity': [quantity], 'BuyPrice': [price],
                                'BuyDatetime': [buy_datetime], 'SellTransactions': [[]], 'PendingQuantity': [quantity],
                                'AverageSellPrice': [0], 'PnL': [0], 'Status': ['Open']})
        self.logs = pd.concat([self.logs, new_log], ignore_index=True)
        return buy_id

    def sell_stock(self, buy_id, quantity, sell_price, date):
        buy_row = self.logs[(self.logs['BuyID'] == buy_id) & (self.logs['Symbol'] == self.symbol)]
        if buy_row.empty:
            return "Buy ID not found."

        sell_datetime = datetime.strptime(date, '%Y-%m-%d')
        pnl = quantity * (sell_price - buy_row['BuyPrice'].values[0])

        # Update the log entry with the sell transaction
        buy_index = buy_row.index[0]
        sell_transaction = {'Quantity': quantity, 'SellPrice': sell_price, 'SellDatetime': sell_datetime, 'PnL': pnl}
        self.logs.at[buy_index, 'SellTransactions'] = self.logs.at[buy_index, 'SellTransactions'] + [sell_transaction]

        # Update pending quantity, average sell price, and cumulative PnL
        self.logs.at[buy_index, 'PendingQuantity'] -= quantity
        self.logs.at[buy_index, 'AverageSellPrice'] = self.calculate_average_sell_price(buy_index)
        self.logs.at[buy_index, 'PnL'] += pnl

        # If all quantity sold, update the status to 'Done'
        if self.logs.at[buy_index, 'PendingQuantity'] == 0:
            self.logs.at[buy_index, 'Status'] = 'Done'

        return f"Stock sold successfully. PnL: {pnl}"

    def calculate_average_sell_price(self, buy_index):
        sell_transactions = self.logs.at[buy_index, 'SellTransactions']
        if not sell_transactions:
            return 0

        total_sell_price = sum(transaction['Quantity'] * transaction['SellPrice'] for transaction in sell_transactions)
        total_quantity = sum(transaction['Quantity'] for transaction in sell_transactions)

        return total_sell_price / total_quantity


# Example usage:
trader = StockTrader(symbol='AAPL', initial_balance=100000)
buy_id_1 = trader.buy_stock(quantity=10, price=50, date='2023-01-01')
sell_result_1 = trader.sell_stock(buy_id=buy_id_1, quantity=5, sell_price=60, date='2023-02-01')


In [79]:
trader.logs

,Symbol,BuyID,Action,Quantity,BuyPrice,BuyDatetime,SellTransactions,PendingQuantity,AverageSellPrice,PnL,Status
0,AAPL,1703415135,Buy,10,50,2023-01-01,"[{'Quantity': 5, 'SellPrice': 60, 'SellDatetim...",0,90.0,400,Done


In [78]:
trader.sell_stock(buy_id=buy_id_1, quantity=5, sell_price=120, date='2023-02-01')

'Stock sold successfully. PnL: 350'